In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
print(torch.__version__)
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset
from torchvision import transforms as T
import random
import cv2
import numpy as np
!pip install pyefd
import pyefd
from google.colab.patches import cv2_imshow
!pip install cairocffi
import cairocffi as cairo
import struct
from struct import unpack
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

1.12.1+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 7.1 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.4.0-py3-none-any.whl size=88775 sha256=10c4edb9e872633d062e749187fe2d25af9917008786e4cd2cb9237a54ae3126
  Stored in directory: /root/.cache/pip/wheels/7a/2b/da/aec872f95d2c24105496ef149a9a576f52daf686f8f2127541
Successfully built cairocffi
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
     |████████████████████████████████| 7.9 MB 27.5 MB/s 
     |████████████████████████████████| 3.5 MB 49.0 MB/s 
     |████████████████████████████████| 2.4 MB 12.0 MB/s 
     |████████████████████████████████| 709 kB 52.3 MB/s 
     |████████████████████████████████| 467

In [8]:
# Env vars
torch.use_deterministic_algorithms(False)

# Const vars
LOAD_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25 GNN all 28.pt'
RAND_SEED = 0
DEVICE = "cuda"

IMG_SIDE = 28
NUM_CLASSES = 25
EPOCHS = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 512
LOSS_FN = nn.CrossEntropyLoss()
FOURIER_ORDER = 10
EDGE_ATTR_DIM = 3

In [9]:
# convert raw vector image to list of raster images, one for each stroke
def vector_to_raster(vector_image, side=IMG_SIDE, line_diameter=16, padding=96, bg_color=(0,0,0), fg_color=(1,1,1)):
  """
  padding and line_diameter are relative to the original 256x256 image.
  """
  
  original_side = 256.
  
  surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, side, side)
  ctx = cairo.Context(surface)
  ctx.set_antialias(cairo.ANTIALIAS_BEST)
  ctx.set_line_cap(cairo.LINE_CAP_ROUND)
  ctx.set_line_join(cairo.LINE_JOIN_ROUND)
  ctx.set_line_width(line_diameter)

  # scale to match the new size
  # add padding at the edges for the line_diameter
  # and add additional padding to account for antialiasing
  total_padding = padding * 2. + line_diameter
  new_scale = float(side) / float(original_side + total_padding)
  ctx.scale(new_scale, new_scale)
  ctx.translate(total_padding / 2., total_padding / 2.)
      
  bbox = np.hstack(vector_image).max(axis=1)
  offset = ((original_side, original_side) - bbox) / 2.
  offset = offset.reshape(-1,1)
  centered = [stroke + offset for stroke in vector_image]

  stroke_rasters = []
  for xv, yv in centered:
    # clear background
    ctx.set_source_rgb(*bg_color)
    ctx.paint()

    # draw strokes, this is the most cpu-intensive part
    ctx.set_source_rgb(*fg_color)        
    ctx.move_to(xv[0], yv[0])
    for x, y in zip(xv, yv):
        ctx.line_to(x, y)
    ctx.stroke()

    data = surface.get_data()
    stroke_raster = np.copy(np.asarray(data)[::4]).reshape(side, side)
    stroke_rasters.append(stroke_raster)

  return stroke_rasters

ELEMENT = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
def get_edges(stroke_angles, stroke_centers, stroke_rasters): 
  adj_1 = []
  adj_2 = []
  edge_info = []
  ANGLE_NORM = 2 * np.pi
  DIST_NORM = np.sqrt(IMG_SIDE**2 + IMG_SIDE**2)
  NUM_STROKES = len(stroke_rasters)
  for i in range(NUM_STROKES):
      stroke_i = cv2.dilate(stroke_rasters[i], ELEMENT).astype(np.float32)
      for j in range(i + 1, NUM_STROKES):
          ij_info = []
          ji_info = []
          adj_1.append(i)
          adj_2.append(j)
          adj_1.append(j)
          adj_2.append(i)
          
          angle_diff = stroke_angles[i] - stroke_angles[j]
          angle_diff_norm_ij = [angle_diff / ANGLE_NORM]
          angle_diff_norm_ji = [-1 * angle_diff / ANGLE_NORM]
          ij_info.append(angle_diff_norm_ij)
          ji_info.append(angle_diff_norm_ji)

          dist = np.sqrt((stroke_centers[i][0] - stroke_centers[j][0])**2 +
                          (stroke_centers[i][1] - stroke_centers[j][1])**2)
          dist_norm = [dist / DIST_NORM]
          ij_info.append(dist_norm)
          ji_info.append(dist_norm)

          temp = stroke_i + stroke_rasters[j]
          if np.amax(temp) > 255:
            ij_info.append([1])
            ji_info.append([1])
          else:
            ij_info.append([0])
            ji_info.append([0])
          
          edge_info.append(ij_info)
          edge_info.append(ji_info)

  edge_indices = torch.LongTensor([adj_1, adj_2])
  edge_attr = torch.FloatTensor(edge_info)
  return edge_indices, edge_attr

means = np.asarray([[1.00000000e+00,  5.14861435e-19,  8.59462640e-19, -4.34051261e-01],
 [ 1.49718473e-02,  1.34727903e-03, -1.84497062e-02, -1.89958526e-02],
 [-4.84771330e-02, -2.51869128e-03, -3.40720294e-03, -1.78159634e-03],
 [ 5.06983955e-03,  1.59704093e-04, -2.62269206e-03,  2.92415198e-03],
 [ 3.21816824e-03, -8.86234804e-04, -6.85097001e-04, -6.91081993e-03],
 [ 6.51948277e-04, -2.99653792e-04, -1.58453633e-03, -1.29729484e-03],
 [ 1.09003765e-03, -3.26678284e-04, -5.82768021e-04, -1.91625458e-03],
 [ 1.02343531e-04, -6.61961033e-06, -2.49310198e-04,  9.74631489e-05],
 [ 1.44199272e-03, -2.94483190e-04, -2.90308786e-04, -1.36471836e-03],
 [ 3.69973782e-05, -2.13514517e-04, -4.55580663e-04, -2.90560724e-04],
 [ 8.84660788e-04, -5.12428761e-05, -5.93233759e-05, -2.28192971e-03],
 [-5.69101248e-05, -1.02366038e-04, -5.02431603e-05, -8.81870114e-07],
 [ 1.17968599e-03, -4.96309536e-05, -1.87174858e-04, -1.10195818e-03],
 [ 1.23174499e-04, -1.05128032e-04, -1.33797737e-04, -1.82618305e-04],
 [ 8.11483486e-04,  6.19275930e-06, -5.14750577e-05, -1.10881976e-03],
 [ 2.15681325e-05, -7.10967890e-05, -2.57723919e-05,  1.09189750e-04],
 [ 8.71751859e-04, -4.29682417e-05,  4.75493672e-06, -1.01930531e-03],
 [ 2.46877824e-05, -3.61324021e-05, -6.78739422e-05, -4.04450630e-05],
 [ 6.08925410e-04,  2.99083480e-05,  2.12464099e-05, -6.85383612e-04],
 [-4.23131458e-05,  2.55885830e-05, -4.04052508e-05,  1.56945537e-05]])

stdevs = np.asarray([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 3.23146768e-01],
 [3.22148277e-01, 2.82445727e-01, 7.27218739e-01, 2.53983631e-01],
 [2.06668586e-01, 1.00220752e-01, 1.44715164e-01, 1.41756867e-01],
 [1.14806784e-01, 1.00487813e-01, 1.18368197e-01, 8.36443685e-02],
 [7.38900712e-02, 5.71741801e-02, 7.00910434e-02, 6.41892768e-02],
 [5.45643073e-02, 4.80276561e-02, 7.12769715e-02, 5.68008593e-02],
 [3.61416521e-02, 3.12290318e-02, 3.98879640e-02, 3.68969178e-02],
 [2.64392042e-02, 2.74921614e-02, 3.47051275e-02, 2.71751311e-02],
 [2.36936809e-02, 2.13939518e-02, 2.61954840e-02, 2.51841936e-02],
 [1.79072047e-02, 1.87636703e-02, 2.40244819e-02, 2.23060269e-02],
 [1.56518788e-02, 1.46354321e-02, 1.84914816e-02, 1.74366632e-02],
 [1.28687817e-02, 1.32499945e-02, 1.60016162e-02, 1.41579594e-02],
 [1.15966920e-02, 1.09186873e-02, 1.25520824e-02, 1.27574098e-02],
 [9.50190175e-03, 9.76960383e-03, 1.19627687e-02, 1.08677787e-02],
 [8.58555829e-03, 8.25293248e-03, 9.90222406e-03, 1.00623277e-02],
 [7.53926101e-03, 7.91677184e-03, 8.51062605e-03, 8.19100746e-03],
 [7.00448435e-03, 6.93656537e-03, 7.70971308e-03, 7.63311218e-03],
 [5.95626758e-03, 6.06239528e-03, 7.00590749e-03, 6.56521224e-03],
 [5.50833888e-03, 5.41913146e-03, 6.40424990e-03, 6.40245777e-03],
 [4.88413673e-03, 5.01467070e-03, 5.56799349e-03, 5.43055859e-03]])
# means = np.asarray([[1.00000000e+00,  8.64704953e-20,  4.23785772e-20, -5.55188263e-01],
#  [-4.21131398e-03, -8.73318590e-04, -7.46787944e-03, -1.16146910e-02],
#  [ 3.45495835e-02, -2.98804448e-04,  1.64286745e-04, -5.45286175e-02],
#  [-5.87583192e-06, -2.53474884e-04, -6.50432941e-05,  3.94554346e-03],
#  [ 5.82578165e-03, -6.83578601e-04,  1.32397937e-04, -7.64260417e-03],
#  [ 3.16973027e-05, -1.02179629e-04, -3.79339444e-04, -4.74683571e-04],
#  [ 1.34949896e-03, -1.36386428e-04,  1.15424560e-04, -1.54791375e-03],
#  [ 6.86240768e-05, -5.95330926e-05,  3.42253351e-05,  5.09141139e-04],
#  [ 1.14870610e-03, -4.14005821e-05,  2.85670771e-05, -5.24566362e-04],
#  [ 1.22019705e-04, -8.23679948e-05, -9.08114311e-05, -8.45407870e-05]])

# stdevs = np.asarray([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 2.28322956e-01],
#  [1.23763294e-01, 1.30004807e-01, 2.44674213e-01, 1.36764044e-01],
#  [8.84229438e-02, 6.22169306e-02, 7.98833353e-02, 8.84160503e-02],
#  [4.84725535e-02, 4.71595999e-02, 5.60523362e-02, 5.11812694e-02],
#  [3.88979383e-02, 3.34160340e-02, 3.58259993e-02, 3.86328886e-02],
#  [2.44812410e-02, 2.43872982e-02, 2.82021738e-02, 2.67411865e-02],
#  [1.97384502e-02, 1.81130527e-02, 2.00832990e-02, 2.08832311e-02],
#  [1.45924819e-02, 1.45891669e-02, 1.58764064e-02, 1.52681562e-02],
#  [1.22818532e-02, 1.17304880e-02, 1.22845295e-02, 1.25736286e-02],
#  [9.61639517e-03, 9.74633403e-03, 1.03337422e-02, 1.01022058e-02]])

# transform functions - take sketch image, return torch tensor of descriptors
def fourier_transform(vector_img, is_test):
  stroke_rasters = vector_to_raster(vector_img)

  # add rotations and translations at test time
  if is_test: 
    stroke_rasters = np.stack(stroke_rasters)
    stroke_rasters = torch.from_numpy(stroke_rasters).float()

    angle = random.random()*60 - 30
    deltaX = random.randint(-3, 3)
    deltaY = random.randint(-3, 3)

    stroke_rasters = T.functional.affine(stroke_rasters,angle,[deltaX, deltaY],1,0,
                                          interpolation=T.InterpolationMode.BILINEAR)
    stroke_rasters = stroke_rasters.numpy().astype(np.uint8)
    stroke_rasters = np.split(stroke_rasters, stroke_rasters.shape[0])
    stroke_rasters = [np.squeeze(a) for a in stroke_rasters]

  stroke_rasters_binary = []
  for raster in stroke_rasters:
    raster_binary = cv2.threshold(raster, 100, 255, cv2.THRESH_BINARY)[1]
    stroke_rasters_binary.append(raster_binary)

  stroke_fourier_descriptors = []
  strokes_to_remove = []
  stroke_angles = []
  stroke_centers = []
  for i, raster in enumerate(stroke_rasters_binary):
    contours = cv2.findContours(raster, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0]

    largest_size = 0
    largest_index = 0
    for k, contour in enumerate(contours):
        if len(contour) > largest_size:
          largest_size = len(contour)
          largest_index = k

    if largest_size > 1:
      contour = np.asarray(contours[largest_index]).squeeze()
      coeffs, transform = pyefd.elliptic_fourier_descriptors(contour, 
                                                          order=FOURIER_ORDER, 
                                                          normalize=True,
                                                          return_transformation=True)
      stroke_angle = transform[1]
      coeffs = (coeffs - means[:FOURIER_ORDER]) / stdevs[:FOURIER_ORDER]
      stroke_fourier_descriptors.append(coeffs.flatten())
      stroke_angles.append(stroke_angle)
      stroke_center = pyefd.calculate_dc_coefficients(contour)
      stroke_centers.append(stroke_center)
    else:
      strokes_to_remove.append(i)

  for i in reversed(strokes_to_remove):
    del stroke_rasters_binary[i]

  edge_indices, edge_attr = get_edges(stroke_angles, stroke_centers,
                                      stroke_rasters_binary)
  stroke_fourier_descriptors = np.stack(stroke_fourier_descriptors)
  stroke_fourier_descriptors = torch.from_numpy(stroke_fourier_descriptors).float()
  return stroke_fourier_descriptors, edge_indices, edge_attr

# helper method to find class based on imgset index
def find_class(idx, num_list):
  class_id = 0
  sum = num_list[class_id]
  while idx >= sum:
    class_id += 1
    sum += num_list[class_id]
  return class_id

# deterministic worker re-seeding
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

# custom dataset for quickdraw
class QuickdrawDataset(Dataset):
  def __init__(self, imgs, nums, is_test):
    self.imgs = imgs
    self.nums = nums
    self.len = sum(nums)
    self.is_test = is_test

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    img = self.imgs[idx]
    x, edge_index, edge_attr = fourier_transform(img, self.is_test)
    y = find_class(idx, self.nums)
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)


# pytorch-geometric GCN
class GCN(torch.nn.Module):
  def __init__(self):
    super(GCN, self).__init__()
    self.conv1 = GCNConv(FOURIER_ORDER * 4, 512)
    self.conv2 = GCNConv(512, 512)
    self.conv3 = GCNConv(512, 512)
    self.conv4 = GCNConv(512, 512)
    self.conv5 = GCNConv(512, 512)
    self.conv6 = GCNConv(512, 512)
    self.conv7 = GCNConv(128, 128)
    self.conv8 = GCNConv(128, 128)
    self.fc1 = nn.Linear(512, 768)
    self.fc2 = nn.Linear(768, 768)
    self.head = nn.Linear(768, NUM_CLASSES)
    self.edge_proj = nn.Linear(EDGE_ATTR_DIM, 1)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()


  def forward(self, data):
    x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

    if EDGE_ATTR_DIM > 1:
      edge_attr = edge_attr.squeeze(dim=2)
    edge_weight = self.edge_proj(edge_attr)
    edge_weight = self.sigmoid(edge_weight)
    x = self.conv1(x, edge_index, edge_weight)
    x = self.relu(x)
    x = self.conv2(x, edge_index, edge_weight)
    x = self.relu(x)
    x = self.conv3(x, edge_index, edge_weight)
    x = self.relu(x)
    x = self.conv4(x, edge_index, edge_weight)
    x = self.relu(x)
    x = global_mean_pool(x, batch)
    x = self.fc1(x)
    x = self.relu(x)
    return self.head(x)


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # put the model in train mode
    total_loss = 0
    total_correct = 0
    # for each batch in the training set compute loss and update model parameters
    for batch, data in enumerate(dataloader):
      data = data.to(DEVICE)
      # Compute prediction and loss
      out = model(data)
      loss = loss_fn(out, data.y)

      # Backpropagation to update model parameters
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # print current training metrics for user
      data, out, loss = data.to("cpu"), out.to("cpu"), loss.to("cpu")
      loss_val = loss.item()
      if batch % 20 == 0:
          current = (batch + 1) * BATCH_SIZE
          print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")

      pred = out.argmax(dim=1, keepdim=True)
      correct = pred.eq(data.y.view_as(pred)).sum().item()
      total_correct += correct
      total_loss += loss_val
      # print(f"train loss: {loss_val:>7f}   train accuracy: {correct / BATCH_SIZE:>7f}   [batch: {batch + 1:>3d}/{(size // BATCH_SIZE) + 1:>3d}]")      
    print(f"\nepoch avg train loss: {total_loss / ((size // BATCH_SIZE) + 1):>7f}   epoch avg train accuracy: {total_correct / size:>7f}")
      
def eval_loop(dataloader, model):
  model.eval()
  size = len(dataloader.dataset)
  with torch.no_grad():
    total_correct = 0
    for data in dataloader:
      data = data.to(DEVICE)
      out = model(data)
      data, out = data.to("cpu"), out.to("cpu")
      pred = out.argmax(dim=1, keepdim=True)
      total_correct += pred.eq(data.y.view_as(pred)).sum().item()

    accuracy = total_correct / size
    return accuracy

In [10]:
# define methods for unpacking Quickdraw .bin files
def unpack_drawing(file_handle):
  file_handle.read(15)
  n_strokes, = unpack('H', file_handle.read(2))
  image = []
  for i in range(n_strokes):
      n_points, = unpack('H', file_handle.read(2))
      fmt = str(n_points) + 'B'
      x = unpack(fmt, file_handle.read(n_points))
      y = unpack(fmt, file_handle.read(n_points))
      image.append((x, y))

  return image


def unpack_drawings(filename):
  imageset = []
  with open(filename, 'rb') as f:
      while True:
          try:
              imageset.append(unpack_drawing(f))
          except struct.error:
              break
  return imageset

train_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset Small/Train/'
test_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset Small/Test/'
train_imgs = []
test_imgs = []
train_nums = []
test_nums = []
list_of_classes = ["arm", "brain", "circle", "ear", "elbow", "eye", 
                   "face", "finger", "foot", "hand", "hexagon", "knee", 
                   "leg", "line", "mouth", "nose", "octagon", 
                   'skull', 'square', 'squiggle', 'star', 'toe', 'tooth', 
                   'triangle', 'zigzag']
# list_of_classes = ["aircraft carrier", "airplane", "alarm clock", "ambulance", 
#                    "angel", "ant", "anvil", "apple", "arm", "asparagus", "axe", 
#                    "backpack", "banana", "bandage", "barn", "baseball bat", 
#                    "baseball", "basket", "basketball", "bathtub", "beach", "bear", 
#                    "beard", "bed", "bee"]

In [11]:
# load dataset
for item in list_of_classes:
  train_folder = train_dir + item + '.bin'
  test_folder = test_dir + item + '.bin'
  train_drawings = unpack_drawings(train_folder)
  train_imgs += train_drawings
  train_nums.append(len(train_drawings))
  test_drawings = unpack_drawings(test_folder)
  test_imgs += test_drawings
  test_nums.append(len(test_drawings))

In [ ]:
DEVICE = "cpu"
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)
g = torch.Generator()
g.manual_seed(RAND_SEED)

def get_confusion_mat(dataloader, model):
  model.eval()
  size = len(dataloader.dataset)
  conf_mat = np.zeros((NUM_CLASSES, NUM_CLASSES))
  with torch.no_grad():
    total_correct = 0
    for data in dataloader:
      data = data.to(DEVICE)
      out = model(data)
      data, out = data.to("cpu"), out.to("cpu")
      pred = out.argmax(dim=1, keepdim=True)
      y = data.y.view_as(pred)
      total_correct += pred.eq(y).sum().item()
      y, pred = y.numpy(), pred.numpy()
      indices = np.concatenate((y, pred), axis=1)
      for i in range(indices.shape[0]):
        conf_mat[indices[i, 0], indices[i, 1]] += 1

    accuracy = total_correct / size
    print(f"test accuracy: {accuracy:>7f}")
    return conf_mat

eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

model = GCN()
checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)

random.seed(RAND_SEED)
conf_mat = get_confusion_mat(dataloader=eval_loader,model=model)
conf_mat_norm = conf_mat * (255 / 100)
cv2_imshow(conf_mat_norm)

diag = np.diagonal(conf_mat)
sums = np.asarray(test_nums)
accuracies = diag / sums
acc_sorted_idxs = np.argsort(accuracies)
for i in range(NUM_CLASSES):
  print(list_of_classes[acc_sorted_idxs[i]] + " accuracy " + str(accuracies[acc_sorted_idxs[i]]))

print("\n")

diag_mat = np.multiply(np.identity(NUM_CLASSES), conf_mat)
conf_mat_nodiag = conf_mat - diag_mat
conf_mat_sorted = np.argsort(conf_mat_nodiag, axis=None)
for i in range(1, 6):
  print("class " + list_of_classes[conf_mat_sorted[-i] // NUM_CLASSES] + " misclassified as " + list_of_classes[conf_mat_sorted[-i] % NUM_CLASSES])

test accuracy: 0.568227


knee accuracy 0.29
nose accuracy 0.35
foot accuracy 0.36
toe accuracy 0.3939393939393939
elbow accuracy 0.43
brain accuracy 0.45
squiggle accuracy 0.48
skull accuracy 0.48
octagon accuracy 0.48
leg accuracy 0.5
arm accuracy 0.52
mouth accuracy 0.54
eye accuracy 0.56
finger accuracy 0.57
face accuracy 0.57
hexagon accuracy 0.58
hand accuracy 0.64
ear accuracy 0.64
tooth accuracy 0.64
zigzag accuracy 0.71
circle accuracy 0.72
star accuracy 0.79
square accuracy 0.8
triangle accuracy 0.81
line accuracy 0.9


class hexagon misclassified as octagon
class skull misclassified as face
class knee misclassified as elbow
class octagon misclassified as hexagon
class face misclassified as eye


In [12]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_fourier_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = GCN()
# checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0
print("\n\n\nFourier order is: "+str(FOURIER_ORDER)+"\n\n\n")

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25 GNN all 28.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_fourier_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)




Fourier order is: 10



Epoch 1

loss: 3.214739  [  512/25000]
loss: 2.645707  [10752/25000]
loss: 2.324461  [20992/25000]

epoch avg train loss: 2.599488   epoch avg train accuracy: 0.225080

-------------------------------

Epoch 2

loss: 2.194233  [  512/25000]
loss: 2.013624  [10752/25000]
loss: 1.999897  [20992/25000]

epoch avg train loss: 2.012316   epoch avg train accuracy: 0.390680

-------------------------------

Epoch 3

loss: 1.844941  [  512/25000]
loss: 1.777017  [10752/25000]
loss: 1.865839  [20992/25000]

epoch avg train loss: 1.815279   epoch avg train accuracy: 0.444640

-------------------------------

Epoch 4

loss: 1.703548  [  512/25000]
loss: 1.707643  [10752/25000]
loss: 1.744934  [20992/25000]

epoch avg train loss: 1.690648   epoch avg train accuracy: 0.476240

-------------------------------

Epoch 5

loss: 1.590774  [  512/25000]
loss: 1.690370  [10752/25000]
loss: 1.585629  [20992/25000]

epoch avg train loss: 1.593448   epoch avg train accuracy: 0.5097

In [7]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_fourier_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = GCN()
# checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0
print("\n\n\nFourier order is: "+str(FOURIER_ORDER)+"\n\n\n")

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25-2 GNN all.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_fourier_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)




Fourier order is: 10



Epoch 1

loss: 3.217189  [  512/24999]
loss: 2.939047  [10752/24999]
loss: 2.737694  [20992/24999]

epoch avg train loss: 2.920878   epoch avg train accuracy: 0.125965

-------------------------------

Epoch 2

loss: 2.635217  [  512/24999]
loss: 2.597538  [10752/24999]
loss: 2.518894  [20992/24999]

epoch avg train loss: 2.558431   epoch avg train accuracy: 0.231849

-------------------------------

Epoch 3

loss: 2.398872  [  512/24999]
loss: 2.387395  [10752/24999]
loss: 2.305570  [20992/24999]

epoch avg train loss: 2.371596   epoch avg train accuracy: 0.293492

-------------------------------

Epoch 4

loss: 2.272254  [  512/24999]
loss: 2.192513  [10752/24999]
loss: 2.228470  [20992/24999]

epoch avg train loss: 2.238240   epoch avg train accuracy: 0.329773

-------------------------------

Epoch 5

loss: 2.265011  [  512/24999]
loss: 2.190886  [10752/24999]
loss: 2.170385  [20992/24999]

epoch avg train loss: 2.130718   epoch avg train accuracy: 0.3581

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_fourier_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = GCN()
# checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0
print("\n\n\nFourier order is: "+str(FOURIER_ORDER)+"\n\n\n")

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25 GNN all.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_fourier_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)




Fourier order is: 10



Epoch 1

loss: 3.215849  [  512/25000]
loss: 2.634784  [10752/25000]
loss: 2.348586  [20992/25000]

epoch avg train loss: 2.574364   epoch avg train accuracy: 0.237440

-------------------------------

Epoch 2

loss: 2.158294  [  512/25000]
loss: 1.917138  [10752/25000]
loss: 1.870495  [20992/25000]

epoch avg train loss: 1.942070   epoch avg train accuracy: 0.416280

-------------------------------

Epoch 3

loss: 1.800991  [  512/25000]
loss: 1.710957  [10752/25000]
loss: 1.780813  [20992/25000]

epoch avg train loss: 1.709600   epoch avg train accuracy: 0.480760

-------------------------------

Epoch 4

loss: 1.551408  [  512/25000]
loss: 1.642351  [10752/25000]
loss: 1.678532  [20992/25000]

epoch avg train loss: 1.577072   epoch avg train accuracy: 0.521800

-------------------------------

Epoch 5

loss: 1.513356  [  512/25000]
loss: 1.618059  [10752/25000]
loss: 1.424187  [20992/25000]

epoch avg train loss: 1.479391   epoch avg train accuracy: 0.5520

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_fourier_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = GCN()
# checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0
print("\n\n\nFourier order is: "+str(FOURIER_ORDER)+"\n\n\n")

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/GNN all subset.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_fourier_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)




Fourier order is: 10



Epoch 1

loss: 3.333715  [  512/28000]
loss: 2.887496  [10752/28000]
loss: 2.398068  [20992/28000]

epoch avg train loss: 2.699190   epoch avg train accuracy: 0.219071

-------------------------------

Epoch 2

loss: 2.211433  [  512/28000]
loss: 2.215790  [10752/28000]
loss: 2.001445  [20992/28000]

epoch avg train loss: 2.082231   epoch avg train accuracy: 0.383679

-------------------------------

Epoch 3

loss: 1.829049  [  512/28000]
loss: 1.877934  [10752/28000]
loss: 1.764905  [20992/28000]

epoch avg train loss: 1.876522   epoch avg train accuracy: 0.432714

-------------------------------

Epoch 4

loss: 1.745137  [  512/28000]
loss: 1.680817  [10752/28000]
loss: 1.715643  [20992/28000]

epoch avg train loss: 1.739501   epoch avg train accuracy: 0.478536

-------------------------------

Epoch 5

loss: 1.685666  [  512/28000]
loss: 1.680699  [10752/28000]
loss: 1.641566  [20992/28000]

epoch avg train loss: 1.643945   epoch avg train accuracy: 0.5058

Dist and angle only

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_fourier_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = GCN()
# checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0
print("\n\n\nFourier order is: "+str(FOURIER_ORDER)+"\n\n\n")

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/GNN DistAngle subset F20-001 wide.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_fourier_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)




Fourier order is: 20



Epoch 1

loss: 3.333235  [  512/28000]
loss: 2.963178  [10752/28000]
loss: 2.456013  [20992/28000]

epoch avg train loss: 2.749457   epoch avg train accuracy: 0.197036

-------------------------------

Epoch 2

loss: 2.265107  [  512/28000]
loss: 2.207243  [10752/28000]
loss: 2.026223  [20992/28000]

epoch avg train loss: 2.119788   epoch avg train accuracy: 0.372964

-------------------------------

Epoch 3

loss: 1.811575  [  512/28000]
loss: 1.890744  [10752/28000]
loss: 1.805849  [20992/28000]

epoch avg train loss: 1.885861   epoch avg train accuracy: 0.432964

-------------------------------

Epoch 4

loss: 1.750515  [  512/28000]
loss: 1.622654  [10752/28000]
loss: 1.679772  [20992/28000]

epoch avg train loss: 1.735321   epoch avg train accuracy: 0.476179

-------------------------------

Epoch 5

loss: 1.648224  [  512/28000]
loss: 1.626344  [10752/28000]
loss: 1.613298  [20992/28000]

epoch avg train loss: 1.615564   epoch avg train accuracy: 0.5100

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_fourier_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = GCN()
# checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0
print("\n\n\nFourier order is: "+str(FOURIER_ORDER)+"\n\n\n")

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/GNN DistAngle subset F10-001 wide.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_fourier_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)




Fourier order is: 10



Epoch 1

loss: 3.333933  [  512/28000]
loss: 2.915837  [10752/28000]
loss: 2.359616  [20992/28000]

epoch avg train loss: 2.662914   epoch avg train accuracy: 0.234500

-------------------------------

Epoch 2

loss: 2.137774  [  512/28000]
loss: 2.158780  [10752/28000]
loss: 1.878676  [20992/28000]

epoch avg train loss: 2.012467   epoch avg train accuracy: 0.405286

-------------------------------

Epoch 3

loss: 1.758918  [  512/28000]
loss: 1.795936  [10752/28000]
loss: 1.771304  [20992/28000]

epoch avg train loss: 1.802730   epoch avg train accuracy: 0.458571

-------------------------------

Epoch 4

loss: 1.696558  [  512/28000]
loss: 1.602885  [10752/28000]
loss: 1.640889  [20992/28000]

epoch avg train loss: 1.675699   epoch avg train accuracy: 0.499500

-------------------------------

Epoch 5

loss: 1.599121  [  512/28000]
loss: 1.603410  [10752/28000]
loss: 1.588686  [20992/28000]

epoch avg train loss: 1.575353   epoch avg train accuracy: 0.5249

In [ ]:
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)
epoch = 10
EPOCHS = 20
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_fourier_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

Epoch 11

loss: 1.104399  [  512/28000]
loss: 1.197649  [10752/28000]
loss: 1.238256  [20992/28000]

epoch avg train loss: 1.155728   epoch avg train accuracy: 0.648214

-------------------------------

Epoch 12

loss: 1.110212  [  512/28000]
loss: 1.108823  [10752/28000]
loss: 1.098974  [20992/28000]

epoch avg train loss: 1.105715   epoch avg train accuracy: 0.658214

-------------------------------

Epoch 13

loss: 1.066953  [  512/28000]
loss: 0.991275  [10752/28000]
loss: 1.045439  [20992/28000]

epoch avg train loss: 1.037860   epoch avg train accuracy: 0.680214

-------------------------------

Epoch 14

loss: 0.931902  [  512/28000]
loss: 0.946194  [10752/28000]
loss: 1.016778  [20992/28000]

epoch avg train loss: 0.978310   epoch avg train accuracy: 0.698179

-------------------------------

Epoch 15

loss: 0.875371  [  512/28000]
loss: 1.024638  [10752/28000]
loss: 0.943012  [20992/28000]

epoch avg train loss: 0.945463   epoch avg train accuracy: 0.706036

------------------

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_fourier_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = GCN()
# checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0
print("\n\n\nFourier order is: "+str(FOURIER_ORDER)+"\n\n\n")

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/GNN DistAngle subset F10-001 wide.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_fourier_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)




Fourier order is: 10



Epoch 1

loss: 3.333168  [  512/28000]
loss: 2.668279  [26112/28000]

epoch avg train loss: 2.905608   epoch avg train accuracy: 0.140143

-------------------------------

Epoch 2

loss: 2.558277  [  512/28000]
loss: 2.304984  [26112/28000]

epoch avg train loss: 2.437220   epoch avg train accuracy: 0.267964

-------------------------------

Epoch 3

loss: 2.162625  [  512/28000]
loss: 2.223273  [26112/28000]

epoch avg train loss: 2.187810   epoch avg train accuracy: 0.346643

-------------------------------

Epoch 4

loss: 2.099264  [  512/28000]
loss: 2.074911  [26112/28000]

epoch avg train loss: 2.044131   epoch avg train accuracy: 0.384393

-------------------------------

Epoch 5

loss: 1.977850  [  512/28000]
loss: 2.000139  [26112/28000]

epoch avg train loss: 1.952012   epoch avg train accuracy: 0.412357

-------------------------------

Epoch 6

loss: 1.968906  [  512/28000]
loss: 1.841549  [26112/28000]

epoch avg train loss: 1.892944   epoch avg 

error: ignored

In [ ]:
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

0.43211861379063954
0.0037479356439321584
